In [9]:
import glob
import numpy as np
from PIL import *
import PIL.Image

In [10]:
filelist = glob.glob('train/glucoma/*.jpg')
glu = np.array([np.array(Image.open(fname)) for fname in filelist])
print(glu.shape)

(28, 2056, 2124, 3)


In [11]:
filelist = glob.glob('train/non/*.jpg')
nonglu = np.array([np.array(Image.open(fname)) for fname in filelist])
print(nonglu.shape)

(252, 2056, 2124, 3)


In [12]:
X = np.concatenate((glu,nonglu),axis=0)
print(X.shape)

(280, 2056, 2124, 3)


In [13]:
gluL = np.array([1 for i in range(28)])
nongluL = np.array([0 for i in range(252)])
y = np.concatenate((gluL,nongluL),axis=0)
print(y.shape)

(280,)


In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.33, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(187, 2056, 2124, 3) (93, 2056, 2124, 3) (187,) (93,)


In [15]:
from keras.models import Sequential
from keras.layers import Activation, Conv2D, MaxPooling2D, Flatten, Dropout,Dense
model = Sequential()

model.add(Conv2D(16, (3,3), padding='same', input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(16, (3,3),padding='valid'))
model.add(Activation('relu'))

model.add(Conv2D(32, (3,3), strides =2))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(32, (3,3), strides =2))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3,3), strides =2))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3,3), strides =2))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3,3), strides =2))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(32))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))


/home/subodh_pushkar/anaconda3/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [16]:
model.output_shape

(None, 1)

In [17]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 2056, 2124, 16)    448       
_________________________________________________________________
activation_1 (Activation)    (None, 2056, 2124, 16)    0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 2054, 2122, 16)    2320      
_________________________________________________________________
activation_2 (Activation)    (None, 2054, 2122, 16)    0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 1026, 1060, 32)    4640      
_________________________________________________________________
activation_3 (Activation)    (None, 1026, 1060, 32)    0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 513, 530, 32)      0         
__________

In [18]:
# from keras import optimizers
# sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
# model.compile(loss='mean_squared_error', optimizer=sgd, metrics = ['accuracy'])
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [20]:
# model.fit(X_train, y_train, batch_size=8, epochs=30, verbose=2, validation_data=(X_test, y_test))
model.fit(X_train, y_train, epochs=10, batch_size=8,validation_data=(X_test, y_test))

Train on 187 samples, validate on 93 samples
Epoch 1/10
187/187 [==============================] - 994s 5s/step - loss: 1.4735 - acc: 0.8984 - val_loss: 1.9064 - val_acc: 0.8817
Epoch 2/10
187/187 [==============================] - 999s 5s/step - loss: 1.5505 - acc: 0.9037 - val_loss: 1.9064 - val_acc: 0.8817
Epoch 3/10
187/187 [==============================] - 992s 5s/step - loss: 1.5517 - acc: 0.9037 - val_loss: 1.9064 - val_acc: 0.8817
Epoch 4/10
187/187 [==============================] - 985s 5s/step - loss: 1.4653 - acc: 0.9091 - val_loss: 1.9064 - val_acc: 0.8817
Epoch 5/10
187/187 [==============================] - 987s 5s/step - loss: 1.4791 - acc: 0.9037 - val_loss: 1.9064 - val_acc: 0.8817
Epoch 6/10
187/187 [==============================] - 983s 5s/step - loss: 1.4653 - acc: 0.9091 - val_loss: 1.9064 - val_acc: 0.8817
Epoch 7/10
187/187 [==============================] - 984s 5s/step - loss: 1.4653 - acc: 0.9091 - val_loss: 1.9064 - val_acc: 0.8817
Epoch 8/10
187/187 [====

In [21]:
score = model.evaluate(X_test, y_test, batch_size=4)
print(score)

93/93 [==============================] - 134s 1s/step
[1.9064414567466919, 0.8817204301075269]


In [22]:
model.predict(X_test, batch_size=8)
model.predict_classes(X_test,batch_size=8)

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
    